# Ajout de la région au jeu de données

In [2]:
import pandas as pd
import requests, json
import urllib.parse

In [3]:
bornes = pd.read_csv('../data/borne_de_recharge.csv')

In [4]:
bornes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50936 entries, 0 to 50935
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nom_operateur           47452 non-null  object 
 1   nom_enseigne            50936 non-null  object 
 2   nom_station             50936 non-null  object 
 3   implantation_station    50936 non-null  object 
 4   adresse_station         50936 non-null  object 
 5   code_insee_commune      36664 non-null  object 
 6   nbre_pdc                50936 non-null  int64  
 7   puissance_nominale      50936 non-null  float64
 8   prise_type_ef           49498 non-null  object 
 9   prise_type_2            49554 non-null  object 
 10  prise_type_combo_ccs    49493 non-null  object 
 11  prise_type_chademo      49493 non-null  object 
 12  prise_type_autre        49493 non-null  object 
 13  gratuit                 49548 non-null  object 
 14  paiement_acte           49571 non-null

In [5]:
api_url = "https://api-adresse.data.gouv.fr/search/?q="
api_url_reverse = "https://api-adresse.data.gouv.fr/reverse/?"

In [6]:
lon = "2.583"
lat = "48.838"
localisation = "lon="+ lon + "&lat=" + lat
r = requests.get(api_url + urllib.parse.quote(localisation))
detail = r.json()


In [7]:
context = detail["features"][0]['properties']['context']
context_split = context.split(',')
code_departement = context_split[0].strip()
nom_departement = context_split[1].strip()
region = context_split[2].strip

In [8]:
nom_departement

'Lozère'

Ok ça fonctionne. Je le fais pour toute la base maintenant.

In [9]:
bornes['code_departement'] = ''
bornes['nom_departement'] = ''
bornes['region'] = ''

In [13]:
for index, row in bornes.iterrows():
    # Récupérer les coordonnées pour la ligne actuelle
    lon, lat = row['consolidated_longitude'], row['consolidated_latitude']
    
    # Appeler l'API pour récupérer les informations de contexte
    url = f'https://api-adresse.data.gouv.fr/reverse/?lon={lon}&lat={lat}'
    response = requests.get(url)
    if response.status_code == 200: 
        detail = response.json()
        if len(detail['features']) > 0:
            context = detail['features'][0]['properties']['context']
            
            # Extraire le code département, le nom du département et la région
            if len(context.split(',')) > 0:
                code_departement = context.split(',')[0].strip()
                if len(context.split(',')) > 1:
                    nom_departement = context.split(',')[1].strip()
                    if len(context.split(',')) > 2:
                        region = context.split(',')[2].strip()
            # Mettre à jour le DataFrame avec les informations récupérées
            bornes.at[index, 'code_departement'] = code_departement
            bornes.at[index, 'nom_departement'] = nom_departement
            bornes.at[index, 'region'] = region
        else:
            print(f"Aucun résultat trouvé pour les coordonnées {lon}, {lat}")
    else:
            print(f"La requête pour les coordonnées {lon}, {lat} a retourné le code d'erreur {response.status_code}")



    
# Enregistrer le DataFrame mis à jour en tant que nouveau fichier CSV
bornes.to_csv('../data/bornes_maj.csv', index=False)

Aucun résultat trouvé pour les coordonnées 5.871263, 48.651079
Aucun résultat trouvé pour les coordonnées 5.871263, 48.651079
Aucun résultat trouvé pour les coordonnées 3.053665, 48.964268
Aucun résultat trouvé pour les coordonnées 1.217755, 49.212071
La requête pour les coordonnées 2.24842, 48.923662 a retourné le code d'erreur 504
La requête pour les coordonnées 2.427408, 48.802789 a retourné le code d'erreur 504
Aucun résultat trouvé pour les coordonnées 5.498871, 47.975185
Aucun résultat trouvé pour les coordonnées 5.497206, 47.977395
La requête pour les coordonnées -2.0288, 48.409 a retourné le code d'erreur 504
La requête pour les coordonnées -2.817927, 48.569214 a retourné le code d'erreur 504
La requête pour les coordonnées -1.982655, 48.486935 a retourné le code d'erreur 504
Aucun résultat trouvé pour les coordonnées -2.75524, 48.34719
Aucun résultat trouvé pour les coordonnées -2.75524, 48.34719
Aucun résultat trouvé pour les coordonnées 6.761461, 44.142858
Aucun résultat tro

In [14]:
bornes.head()

,nom_operateur,nom_enseigne,nom_station,implantation_station,adresse_station,code_insee_commune,nbre_pdc,puissance_nominale,prise_type_ef,prise_type_2,...,date_mise_en_service,date_maj,last_modified,consolidated_longitude,consolidated_latitude,prix_kWh,prix_session,code_departement,nom_departement,region
0,Izivia,ARCACHON,ARCACHON - AIGUILLON,Voirie,RUE DE LA PÉCHERIE,33009,1,7.0,False,True,...,2021-05-19,2023-02-27,2023-02-27T01:50:25Z,-1.146988,44.656649,NaN,NaN,33,Gironde,Nouvelle-Aquitaine
1,Izivia,ARCACHON,ARCACHON - PEREIRE,Voirie,AVENUE DU PARC PÉREIRE,33009,1,7.0,False,True,...,2021-04-15,2023-02-27,2023-02-27T01:50:25Z,-1.193047,44.658496,NaN,NaN,33,Gironde,Nouvelle-Aquitaine
2,Izivia,ARCACHON,ARCACHON - AIGUILLON,Voirie,RUE DE LA PÉCHERIE,33009,1,7.0,False,True,...,2021-05-04,2023-02-27,2023-02-27T01:50:25Z,-1.146988,44.656649,NaN,NaN,33,Gironde,Nouvelle-Aquitaine
3,Izivia,ARCACHON,ARCACHON - AIGUILLON,Voirie,RUE DE LA PÉCHERIE,33009,1,7.0,True,True,...,2021-05-04,2023-02-27,2023-02-27T01:50:25Z,-1.146988,44.656649,NaN,NaN,33,Gironde,Nouvelle-Aquitaine
4,Izivia,ARCACHON,ARCACHON - AIGUILLON,Voirie,RUE DE LA PÉCHERIE,33009,1,7.0,False,True,...,2021-05-04,2023-02-27,2023-02-27T01:50:25Z,-1.146988,44.656649,NaN,NaN,33,Gironde,Nouvelle-Aquitaine


## gestion des non trouvés

In [19]:
bornes_region_vide = bornes[bornes['region']==""]

In [20]:
bornes_region_vide.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 883 entries, 139 to 49807
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nom_operateur           855 non-null    object 
 1   nom_enseigne            883 non-null    object 
 2   nom_station             883 non-null    object 
 3   implantation_station    883 non-null    object 
 4   adresse_station         883 non-null    object 
 5   code_insee_commune      614 non-null    object 
 6   nbre_pdc                883 non-null    int64  
 7   puissance_nominale      883 non-null    float64
 8   prise_type_ef           873 non-null    object 
 9   prise_type_2            873 non-null    object 
 10  prise_type_combo_ccs    873 non-null    object 
 11  prise_type_chademo      873 non-null    object 
 12  prise_type_autre        873 non-null    object 
 13  gratuit                 875 non-null    object 
 14  paiement_acte           875 non-null  

In [26]:
import folium

m = folium.Map(location=[46.227638, 2.213749], zoom_start=6)

for index, row in bornes_region_vide.iterrows():
    # Récupérer la latitude et la longitude du point
    lat = row['consolidated_latitude']
    lon = row['consolidated_longitude']
    # Ajouter un marqueur à la carte pour le point
    folium.Marker([lat, lon]).add_to(m)

# Afficher la carte
m

In [27]:
for index, row in bornes_region_vide.iterrows():
    # Récupérer les coordonnées pour la ligne actuelle
    lon, lat = row['consolidated_longitude'], row['consolidated_latitude']
    
    # Appeler l'API pour récupérer les informations de contexte
    url = f'https://api-adresse.data.gouv.fr/reverse/?lon={lon}&lat={lat}'
    response = requests.get(url)
    if response.status_code == 200: 
        detail = response.json()
        if len(detail['features']) > 0:
            context = detail['features'][0]['properties']['context']
            
            # Extraire le code département, le nom du département et la région
            if len(context.split(',')) > 0:
                code_departement = context.split(',')[0].strip()
                if len(context.split(',')) > 1:
                    nom_departement = context.split(',')[1].strip()
                    if len(context.split(',')) > 2:
                        region = context.split(',')[2].strip()
            # Mettre à jour le DataFrame avec les informations récupérées
            bornes_region_vide.at[index, 'code_departement'] = code_departement
            bornes_region_vide.at[index, 'nom_departement'] = nom_departement
            bornes_region_vide.at[index, 'region'] = region
        else:
            print(f"Aucun résultat trouvé pour les coordonnées {lon}, {lat}")
    else:
            print(f"La requête pour les coordonnées {lon}, {lat} a retourné le code d'erreur {response.status_code}")



    
# Enregistrer le DataFrame mis à jour en tant que nouveau fichier CSV
bornes_region_vide.to_csv('../data/bornes_region_vide.csv', index=False)

Aucun résultat trouvé pour les coordonnées 5.871263, 48.651079
Aucun résultat trouvé pour les coordonnées 5.871263, 48.651079
Aucun résultat trouvé pour les coordonnées 3.053665, 48.964268
Aucun résultat trouvé pour les coordonnées 1.217755, 49.212071
Aucun résultat trouvé pour les coordonnées 5.498871, 47.975185
Aucun résultat trouvé pour les coordonnées 5.497206, 47.977395
Aucun résultat trouvé pour les coordonnées -2.75524, 48.34719
Aucun résultat trouvé pour les coordonnées -2.75524, 48.34719
Aucun résultat trouvé pour les coordonnées 6.761461, 44.142858
Aucun résultat trouvé pour les coordonnées 6.829526, 43.999862
Aucun résultat trouvé pour les coordonnées 6.829526, 43.999862
Aucun résultat trouvé pour les coordonnées 6.761461, 44.142858
Aucun résultat trouvé pour les coordonnées 1.950778, 45.291775
Aucun résultat trouvé pour les coordonnées 1.950778, 45.291775
Aucun résultat trouvé pour les coordonnées 2.456431, 45.622261
Aucun résultat trouvé pour les coordonnées -0.85208, 44.3

SyntaxError: invalid syntax (3995831270.py, line 1)